In [1]:
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data_preprocessed_sort.csv')

In [10]:
df.columns

Index(['customer_id', 'age', 'gender_cd', 'region_size', 'foreign_flg',
       'entrepreneur_flg', 'auto_flg', 'traveler_flg', 'partnership_flg',
       'child_amt', 'realty_type', 'segment_cd', 'bundle_nm',
       'confirmed_income', 'predicted_income', 'product_cd', 'functional_cd',
       'finish_dttm', 'state_talk_time_sec', 'wo_hit_status_result_desc',
       'wo_hit_status_reason_desc', 'employee_id', 'emp_age', 'emp_gender_cd',
       'emp_children_cnt', 'emp_work_months_cnt', 'emp_citizenship_country_cd',
       'emp_marital_status_cd', 'day_of_week', 'holiday_flg', 'approval_flg',
       'bundle_flg', 'segment_region', 'preapprove_flg', 'child_flg'],
      dtype='object')

<h3>ИВ: Что влияет на восприимчивость клиентов к предложениям о кредите?
Гипотеза: Люди без подписки более восприимчивы к предложениям банка о кредите по сравнению с людьми, имеющими подписку Pro.</h3>

In [33]:
df = pd.read_csv('data_preprocessed_sort.csv')

In [34]:
df = df[df['functional_cd'] == 'Preapprove']

In [9]:
df.groupby(['functional_cd', 'product_cd'])['product_cd'].count()

functional_cd  product_cd        
Activation     Car Loan                 1
               Cash Loan               86
Afterfilling   Car Loan                 3
               Casco                  220
               Cash Loan              100
               Credit Cards           444
               Debit Cards            131
               Investment              14
               MVNO                   826
               Mortgage Insurance      34
               Secured Loan            13
Agreement      Car Loan               135
               Cash Loan              242
               Secured Loan            43
Autoleads      Investment              36
BAF            Debit Cards             62
Cold           Car Loan                 1
               Casco                   16
               Debit Cards            465
               MVNO                   623
               Mortgage Insurance     261
               T-Auto                   2
               Travel Avia              3


In [36]:
df.groupby('child_flg')['approval_flg'].agg(mean='mean', count='count')

,mean,count
child_flg,,
0,0.032594,6627
1,0.042049,1308


In [37]:
no = df[df['bundle_nm'] == '0']['approval_flg']
pro = df[df['bundle_nm'] == 'Pro']['approval_flg']
premium = df[df['bundle_nm'] == 'Premium']['approval_flg']

In [38]:
st.mannwhitneyu(no, pro)[1].round(3)

np.float64(0.026)

In [39]:
df = df[(df['bundle_nm'] == '0') | (df['bundle_nm'] == 'Pro')]
df.bundle_nm = df.bundle_nm.map({'Pro': 1, '0': 0})

In [40]:
X = df[['bundle_nm', 'age', 'gender_cd', 'child_flg']]
y = df['approval_flg']

X = sm.add_constant(X)

probit_model = sm.Probit(y, X)
probit_result = probit_model.fit()


print(probit_result.summary())

Optimization terminated successfully.
         Current function value: 0.148314
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:           approval_flg   No. Observations:                 7830
Model:                         Probit   Df Residuals:                     7825
Method:                           MLE   Df Model:                            4
Date:                Sat, 30 Nov 2024   Pseudo R-squ.:                 0.01121
Time:                        17:48:16   Log-Likelihood:                -1161.3
converged:                       True   LL-Null:                       -1174.5
Covariance Type:            nonrobust   LLR p-value:                 2.722e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1291      0.105    -20.230      0.000      -2.335      -1.923
bundle_nm     -0.1846      0.